In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf


In [ ]:
data_dir = '/content/drive/My Drive/ML Assinment 2/assinment 2 part 2/Project/aclImdb/'

# Function to load dataset from a directory
def load_data_from_dir(data_dir, sentiment):
    sentences = []
    sentiments = []
    for file_name in os.listdir(data_dir):
        if file_name.endswith(".txt"):
            with open(os.path.join(data_dir, file_name), 'r', encoding='utf-8') as file:
                sentences.append(file.read())
                sentiments.append(sentiment)
    return sentences, sentiments

# Load train and test data
train_pos_dir = os.path.join(data_dir, 'train/pos')
train_neg_dir = os.path.join(data_dir, 'train/neg')
test_pos_dir = os.path.join(data_dir, 'test/pos')
test_neg_dir = os.path.join(data_dir, 'test/neg')

train_pos_sentences, train_pos_sentiments = load_data_from_dir(train_pos_dir, 1)
train_neg_sentences, train_neg_sentiments = load_data_from_dir(train_neg_dir, 0)
test_pos_sentences, test_pos_sentiments = load_data_from_dir(test_pos_dir, 1)
test_neg_sentences, test_neg_sentiments = load_data_from_dir(test_neg_dir, 0)

# Combine the data
train_sentences = train_pos_sentences + train_neg_sentences
train_sentiments = train_pos_sentiments + train_neg_sentiments
test_sentences = test_pos_sentences + test_neg_sentiments
test_sentiments = test_pos_sentiments + test_neg_sentiments

# Convert to DataFrame for easier manipulation
df_train = pd.DataFrame({'sentence': train_sentences, 'label': train_sentiments})
df_test = pd.DataFrame({'sentence': test_sentences, 'label': test_sentiments})


# Function to load dataset from a directory
def load_data_from_dir(data_dir, sentiment):
    sentences = []
    sentiments = []
    for file_name in os.listdir(data_dir):
        if file_name.endswith(".txt"):
            with open(os.path.join(data_dir, file_name), 'r', encoding='utf-8') as file:
                sentences.append(file.read())
                sentiments.append(sentiment)
    return sentences, sentiments

# Load train and test data
train_pos_dir = os.path.join(data_dir, 'train/pos')
train_neg_dir = os.path.join(data_dir, 'train/neg')
test_pos_dir = os.path.join(data_dir, 'test/pos')
test_neg_dir = os.path.join(data_dir, 'test/neg')

train_pos_sentences, train_pos_sentiments = load_data_from_dir(train_pos_dir, 1)
train_neg_sentences, train_neg_sentiments = load_data_from_dir(train_neg_dir, 0)
test_pos_sentences, test_pos_sentiments = load_data_from_dir(test_pos_dir, 1)
test_neg_sentences, test_neg_sentiments = load_data_from_dir(test_neg_dir, 0)

# Combine the data
train_sentences = train_pos_sentences + train_neg_sentences
train_sentiments = train_pos_sentiments + train_neg_sentiments
test_sentences = test_pos_sentences + test_neg_sentiments
test_sentiments = test_pos_sentiments + test_neg_sentiments

# Convert to DataFrame for easier manipulation
df_train = pd.DataFrame({'sentence': train_sentences, 'label': train_sentiments})
df_test = pd.DataFrame({'sentence': test_sentences, 'label': test_sentiments})


In [ ]:
labeled_dir = os.path.join(data_dir, 'labeled')
sentences = []
aspects = []
sentiments = []

# Read labeled data
for file_name in os.listdir(labeled_dir):
    if file_name.endswith(".txt"):
        with open(os.path.join(labeled_dir, file_name), 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split('",')
                sentence = parts[0][1:]
                aspect_sentiment = parts[1].split(',')
                aspect = aspect_sentiment[0]
                sentiment = aspect_sentiment[1]

                sentences.append(sentence)
                aspects.append(aspect)
                sentiments.append(sentiment)

# Convert to DataFrame for easier manipulation
df = pd.DataFrame({
    'Sentence': sentences,
    'Aspect': aspects,
    'Sentiment': sentiments
})

# Preprocess the labels
le = LabelEncoder()
df['Sentiment'] = le.fit_transform(df['Sentiment'])

# Create aspect embedding
aspect_dict = {
    'Writing': 0,
    'Acting': 1,
    'Directing': 2,
    'Cinematography': 3,
    'Production': 4
}
df['Aspect'] = df['Aspect'].apply(lambda x: aspect_dict[x])
df['Aspect'] = df['Aspect'].astype(str)

# Combine sentence and aspect for training
df['Combined'] = df['Aspect'] + " [SEP] " + df['Sentence']

# Split the data
train, test = train_test_split(df, test_size=0.2, random_state=42)
train_examples = train.apply(lambda x: InputExample(guid=None, text_a=x['Combined'], text_b=None, label=x['Sentiment']), axis=1)
test_examples = test.apply(lambda x: InputExample(guid=None, text_a=x['Combined'], text_b=None, label=x['Sentiment']), axis=1)


In [ ]:
# Function to convert DataFrame to InputFeatures
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = []

    for e in examples:
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length,
            pad_to_max_length=True,
            return_token_type_ids=True,
            return_attention_mask=True,
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"], input_dict["token_type_ids"], input_dict["attention_mask"])

        features.append(
            InputFeatures(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        (
            {
                "input_ids": tf.int32,
                "attention_mask": tf.int32,
                "token_type_ids": tf.int32,
            },
            tf.int64,
        ),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


In [ ]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Convert examples to TensorFlow dataset
train_dataset = convert_examples_to_tf_dataset(list(train_examples), tokenizer)
test_dataset = convert_examples_to_tf_dataset(list(test_examples), tokenizer)

train_dataset = train_dataset.shuffle(100).batch(32).repeat(2)
test_dataset = test_dataset.batch(32)


In [ ]:
# Load the BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Model summary
model.summary()


In [ ]:
# Train the model
history = model.fit(train_dataset, epochs=2, validation_data=test_dataset)


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Test Accuracy: {accuracy * 100:.2f}%')
